In [2]:
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.3f}'.format

### 피처 엔지니어링      

가설 검정에서 만들었던 모든 피쳐(변수)를 정리해 보겠습니다. 이제 예측 모델링을 위한 데이터가 준비되었습니다.

In [4]:
mdl_data = pd.read_pickle('mdl_data.pkl')
mdl_data.head()
print(mdl_data.index.min(), mdl_data.index.max())


2021-01-04 00:00:00 2022-03-24 00:00:00


<br> 가설검정에서 만들었던 모든 피쳐를 정리합니다. 단, *"5일 이동평균선이 종가보다 위에 있다"* 는 유의미하지 않았으므로 제외입니다.

In [5]:
kosdaq_list = pd.read_pickle('kosdaq_list.pkl')

feature_all = pd.DataFrame()

for code, sector in zip(kosdaq_list['code'], kosdaq_list['sector']):

    data = mdl_data[mdl_data['code']==code].sort_index().copy()
    
    
    # 가격변동성이 크고, 거래량이 몰린 종목이 주가가 상승한다
    data['price_mean'] = data['close'].rolling(20).mean()
    data['price_std'] = data['close'].rolling(20).std(ddof=0)
    data['price_z'] = (data['close'] - data['price_mean'])/data['price_std']    
    data['volume_mean'] = data['volume'].rolling(20).mean()
    data['volume_std'] = data['volume'].rolling(20).std(ddof=0)
    data['volume_z'] = (data['volume'] - data['volume_mean'])/data['volume_std']      
    
    
    # 위꼬리가 긴 양봉이 자주발생한다.
    data['positive_candle'] = (data['close'] > data['open']).astype(int) # 양봉
    data['high/close'] = (data['positive_candle']==1)*(data['high']/data['close'] > 1.1).astype(int) # 양봉이면서 고가가 종가보다 높게 위치
    data['num_high/close'] =  data['high/close'].rolling(20).sum()
    data['long_candle'] = (data['positive_candle']==1)*(data['high']==data['close'])*\
    (data['low']==data['open'])*(data['close']/data['open'] > 1.2).astype(int) # 장대 양봉을 데이터로 표현
    data['num_long'] =  data['long_candle'].rolling(60).sum() # 지난 20 일 동안 장대양봉의 갯 수
    
    
     # 거래량이 종좀 터지며 매집의 흔적을 보인다   
    data['volume_mean'] = data['volume'].rolling(60).mean()
    data['volume_std'] = data['volume'].rolling(60).std()
    data['volume_z'] = (data['volume'] - data['volume_mean'])/data['volume_std'] # 거래량은 종목과 주가에 따라 다르기 떄문에 표준화한 값이 필요함
    data['z>1.96'] = (data['close'] > data['open'])*(data['volume_z'] > 1.65).astype(int) # 양봉이면서 거래량이 90%신뢰구간을 벗어난 날
    data['num_z>1.96'] =  data['z>1.96'].rolling(60).sum()  # 양봉이면서 거래량이 90%신뢰구간을 벗어난 날을 카운트
    
    # 주가지수보다 더 좋은 수익율을 보여준다
    data['num_win_market'] = data['win_market'].rolling(60).sum() # 주가지수 수익율이 1 보다 작을 때, 종목 수익율이 1 보다 큰 날 수
    data['pct_win_market'] = (data['return']/data['kosdaq_return']).rolling(60).mean() # 주가지수 수익율 대비 종목 수익율
    
    
    # 동종업체 수익률보다 더 좋은 수익율을 보여준다.           
    data['return_mean'] = data['return'].rolling(60).mean() # 종목별 최근 60 일 수익율의 평균
    data['sector'] = sector 
       
    data['max_close']  = data[['close_r1','close_r2','close_r3','close_r4','close_r5']].max(axis=1) # 5 영업일 종가 수익율 중 최고 값   
    data['mean_close']  = data[['close_r1','close_r2','close_r3','close_r4','close_r5']].mean(axis=1) # 5 영업일 종가 수익율 중 최고 값   
    data['min_close']  = data[['close_r1','close_r2','close_r3','close_r4','close_r5']].min(axis=1) # 5 영업일 종가 수익율 중 최저 값   

    data = data[(data['price_std']!=0) & (data['volume_std']!=0)] 
    
    feature_all = pd.concat([data, feature_all], axis=0)

feature_all['sector_return'] = feature_all.groupby(['sector', feature_all.index])['return'].transform(lambda x: x.mean()) # 섹터의 평균 수익율 계산
feature_all['return over sector'] = (feature_all['return']/feature_all['sector_return']) # 섹터 평균 수익률 대비 종목 수익률 계산
feature_all.dropna(inplace=True) # Missing 값 있는 행 모두 제거


# 최종 피처 및 수익률 데이터만으로 구성
feature_all = feature_all[['code', 'sector','return','kosdaq_return','price_z','volume_z','num_high/close','num_long','num_z>1.96','num_win_market','pct_win_market','return over sector','max_close','mean_close','min_close']]
feature_all.to_pickle('feature_all.pkl')  

<br> 이제 모델링을 위한 데이터 준비가 끝났습니다. 간단한 프로파일을 뽑아봅니다. 평균과 표준편차 값을 보고, 피처들이 제대로 생성되었는 지 확인합니다. 그리고 price_z 와 volum_z 는 같이 분석했을 때 유의미했다는 사실을 기억하면 좋겠습니다.

In [3]:
feature_all = pd.read_pickle('feature_all.pkl') 
feature_all.describe(percentiles=[0.05, 0.1, 0.9, 0.95]).style.set_table_attributes('style="font-size: 10px"')

,return,kosdaq_return,price_z,volume_z,num_high/close,num_long,num_z>1.96,num_win_market,pct_win_market,return over sector,max_close,mean_close,min_close
count,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000,330186.000000
mean,1.000419,0.999977,-0.105830,-0.057986,0.125996,0.017118,1.947639,7.555130,1.000531,1.000000,1.032886,1.001175,0.970123
std,0.034832,0.012915,1.315364,1.062726,0.386956,0.131749,2.015173,2.851991,0.004255,0.029940,0.071121,0.053219,0.049497
min,0.326241,0.962748,-4.358899,-2.032113,0.000000,0.000000,0.000000,0.000000,0.975515,0.378740,0.699957,0.508919,0.416061
5%,0.954712,0.977728,-2.082442,-0.867949,0.000000,0.000000,0.000000,3.000000,0.995183,0.963267,0.969313,0.931217,0.883943
10%,0.967085,0.983467,-1.727909,-0.721448,0.000000,0.000000,0.000000,4.000000,0.996338,0.973544,0.981090,0.949527,0.910898
50%,1.000000,1.001379,-0.223484,-0.311405,0.000000,0.000000,1.000000,7.000000,0.999988,0.998501,1.016949,0.997990,0.977330
90%,1.033109,1.014822,1.671378,0.685147,1.000000,0.000000,5.000000,11.000000,1.005305,1.026457,1.095890,1.052602,1.016327
95%,1.050749,1.021458,2.126138,1.677728,1.000000,0.000000,6.000000,12.000000,1.007807,1.042238,1.142682,1.080308,1.031542
max,1.300461,1.045516,4.358899,7.616867,5.000000,2.000000,15.000000,22.000000,1.043156,1.399210,3.702674,2.346198,1.300461
